In [1]:
# rl learning 

In [2]:
#!pip install tensorflow
#!pip install gym
#!pip install keras
#!pip install keras-rl2

In [3]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [4]:
class ShowerEnv(Env):
    
    
    def __init__(self):
        
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]),dtype=float)
        
        # Set start temp
        self.state = 38 + random.randint(-3,3)  # = +-3
        
        # Set shower length
        self.shower_length = 60
        
        
        
    def step(self, action):
        
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info
    
    
    '''
    def render(self):
        # Implement viz
        pass
    '''
    def reset(self):
        
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3) 
        
        # Reset shower time
        self.shower_length = 60 
        
        return self.state
    
    
    def get_environment_info(self):
        
        print("\nStates : ")
        print(f'object : {self.observation_space}')
        print(f'shape : {self.observation_space.shape}')
        print(f'sample : {self.observation_space.sample()}')
        
        print("\nAction : ")
        print(f'object : {self.action_space}')
        print(f'shape : {self.action_space.shape}')
        print(f'n : {self.action_space.n}')
        print(f'sample : {self.action_space.sample()}')

In [5]:
env = ShowerEnv()

/home/eguimard/.local/lib/python3.10/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float64
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [6]:
env.observation_space.sample()

array([23.58417942])

In [7]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-28
Episode:2 Score:-24
Episode:3 Score:-18
Episode:4 Score:-60
Episode:5 Score:-38
Episode:6 Score:-40
Episode:7 Score:-26
Episode:8 Score:-24
Episode:9 Score:-22
Episode:10 Score:-60


# CREATE DEEP LEARNING MODEL

In [20]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [21]:

print(env.get_environment_info())



States : 
object : Box(0.0, 100.0, (1,), float64)
shape : (1,)
sample : [82.29880877]

Action : 
object : Discrete(3)
shape : ()
n : 3
sample : 2
None


In [22]:
states = env.observation_space.shape
actions = env.action_space.n

In [24]:
states

(1,)

In [25]:
actions

3

In [26]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [27]:
model = build_model(states, actions)

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 24)                48        
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


# 3 BUILD AGENT with KEARS-RL

In [29]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [30]:
def build_agent(model, actions):
    print(f'AAA : {actions}')
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    agent = DQNAgent(model=model, memory=memory, policy=policy,
                     nb_actions=actions, nb_steps_warmup=10,
                     target_model_update=1e-2)
    return agent

In [31]:
agent = build_agent(model, actions)
agent.compile(Adam(learning_rate=1e-3), metrics=['mae'])
agent.fit(env, nb_steps=50000, visualize=False, verbose=1)

AAA : 3


2022-09-27 11:08:43.398408: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 6:54 - reward: 1.0000

/home/eguimard/.local/lib/python3.10/site-packages/keras/engine/training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/home/eguimard/.local/lib/python3.10/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


 8647/10000 [========================>.....] - ETA: 6s - reward: -0.5180done, took 38.582 seconds


In [ ]:
scores = agent.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

In [ ]:

_ = agent.test(env, nb_episodes=15, visualize=True)